In [1]:
from numpy import array, exp 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import openturns as ot
from scipy.optimize import curve_fit 
from scipy.stats import genhyperbolic
from scipy.stats import levy_stable
from scipy.stats import norm
from scipy.stats import cumfreq
from scipy.stats import kstest
from scipy.stats import ks_2samp
from scipy.stats import cramervonmises_2samp
from scipy.stats import anderson_ksamp

dataset = pd.read_csv("../data/log_return_data.csv")
dataset

,log_return_vkco,log_return_tcs_il,log_return_poly_il,log_return_five_il
0,-0.012110,-0.004162,-0.003712,-0.013377
1,-0.035292,0.000000,-0.029352,-0.024070
2,0.011583,0.002083,-0.013279,0.007894
3,-0.006740,0.004154,0.025964,-0.002916
4,-0.009709,0.015425,-0.010984,-0.007034
...,...,...,...,...
248,-0.007648,0.018653,0.030041,0.010193
249,0.019012,0.044184,0.029452,-0.006216
250,0.018657,0.038540,0.008584,0.036732
251,0.046940,0.035287,0.042393,0.000000


In [2]:
#Делим интервал значений случ. вел на bins подинтервалов. Вычисляем количество попаданий в эти подинтервалы
#Фактически имитируем набор данных для plot.hist()
def get_intervals(dataframe, column, bins):
    intervals = []
    data = dataframe[column]
    ln = len(data)
    max_value = max(data)
    min_value = min(data)
    interval_length = (max_value - min_value)
    for i in range(0, bins):
        s = 0
        a = min_value + (i)*interval_length/bins
        b = min_value + (i+1)*interval_length/bins
        for k in range(0, len(data)):
            if (data[k]>= a and data[k]<b):
                s = s + 1
        #intervals.append(s/(ln-1))
        intervals.append(s)
    x_interavals = np.arange(min_value, max_value, abs(max_value- min_value)/bins)
    return intervals, x_interavals, min_value, max_value


In [3]:
def get_rvs_intervals(data, bins, x_data):
    intervals = []
    max_value = max(x_data)
    min_value = min(x_data)
    interval_length = (max_value - min_value)
    for i in range(0, bins):
        s = 0
        a = min_value + (i)*interval_length/bins
        b = min_value + (i+1)*interval_length/bins
        for k in range(0, len(data)):
            if (data[k]>= a and data[k]<b):
                s = s + 1
        intervals.append(s)
    norm_intervals = [x/sum(intervals) for x in intervals]
    return norm_intervals

In [4]:
#Определяем приближающую функцию - Нормальное распределение
def norm_mapping(values_x, a, b): 
    #return 1/(math.sqrt(2*math.pi)*b) * exp(-1/2*((values_x-a)/b)**2)
    return norm.pdf(values_x,a,b)
    

In [5]:
#Определяем приближающую функцию - Гиперболическое распределение
def hyper_mapping(values_x, p, a, b, mu, delta):
    #genhyperbolic.pdf(x, p, a, b, loc, scale)  scale = delta, loc = mu, p = lambda, a = alpha delta, b = beta delta
    if (a>b):
        return genhyperbolic.pdf(values_x, p, a, b, mu, delta)
    else:
        return 0

In [6]:
#Определяем приближающую функцию - Устойчивое распределение
def stable_mapping(values_x, alpha, beta, gamma, mu):
    return levy_stable.pdf(values_x, alpha, beta, gamma, mu)

In [7]:
#Определяем приближающую функцию - Устойчивое распределение
def stable_mapping(values_x, alpha, beta, gamma, mu):
    return levy_stable.pdf(values_x, alpha, beta, gamma, mu)


In [8]:
def test(x, a, b, c, d, e, f):
    return a+b*x+c*x**2+d*x**3+e*x**4+f*x**5


In [9]:
#Читаем файл с данными
dataset = pd.read_csv("../data/log_return_data.csv")

#Выполняем обработку исходных данных
bins = 25
compName = 'log_return_five_il' #'log_return_five_il' log_return_vkco	log_return_tcs_il	log_return_poly_il gazpr_futures
print(compName)
readData = get_intervals(dataset, compName, bins)  
y_data, x_data, min_value, max_value = readData[0], readData[1], readData[2], readData[3]
x_fit = np.arange(min_value, max_value, abs(max_value- min_value)/400)
y_data1 = [x/sum(y_data) for x in y_data]

#Строим эмпирическую функцию распределения
emp_data = cumfreq(dataset[compName], numbins=bins, defaultreallimits=None, weights=None)


log_return_five_il


In [10]:
def get_emp_data(dataframe, columns, bins):
    #Выполняем обработку исходных данных
    readData = get_intervals(dataset, columns, bins)  
    y_data, x_data, min_value, max_value = readData[0], readData[1], readData[2], readData[3]
    x_fit = np.arange(min_value, max_value, abs(max_value- min_value)/400)
    y_data1 = [x/sum(y_data) for x in y_data]

    #Строим эмпирическую функцию распределения
    emp_data = cumfreq(dataset[columns], numbins=bins, defaultreallimits=None, weights=None)
    return emp_data, y_data1, x_fit, x_data, y_data


In [11]:
def closer_finding_functions_hyperbolic(x_data, y_data, data, column):
    #Производим приближение - Гиперболическое распределение
    params, _  = curve_fit(hyper_mapping, x_data, y_data) 
    p, a, b, mu, delta = params[0], params[1], params[2], params[3], params[4]
    y_fit = genhyperbolic.pdf(x_fit, p, a, b, mu, delta) 
    
    #Считаем pvalue - Гиперболическое распределение
    r = genhyperbolic.rvs(p, a, b, mu, delta, size=1000, random_state=None)
    #Выполняем нормировку набора случайных величин
    norm_rvs_data = get_rvs_intervals(r, bins, x_data)
    norm_rvs_data1 = [x/sum(norm_rvs_data) for x in norm_rvs_data]

    return params, r, y_fit, norm_rvs_data

In [12]:
#Производим приближение - Устойчивое распределение
def closer_finding_functions_stable(x_data, y_data):
    
    params, _  = curve_fit(stable_mapping, x_data, y_data, bounds=([1.0001, -1., -1., 0.],[2, 1., 6., 6.])) 
    # args, _  = curve_fit(stable_mapping, x_data, y_data) 
    alpha, beta , gamma, mu = params[0], params[1], params[2], params[3]
    y_fit = levy_stable.pdf(x_fit, alpha, beta, gamma, mu)

    #Считаем pvalue - Устойчивое распределение
    r = levy_stable.rvs(alpha, beta, gamma, mu, size=1000, random_state=None)
    #Выполняем нормировку набора случайных величин
    norm_rvs_data = get_rvs_intervals(r, bins, x_data)
    norm_rvs_data = [x/sum(norm_rvs_data) for x in norm_rvs_data]
    return params, r, y_fit, norm_rvs_data


In [13]:
#Производим приближение - Устойчивое распределение
def closer_finding_functions_stable(x_data, data, columns):
    params = levy_stable._fitstart(data[columns])
    alpha, beta , gamma, mu = params[0], params[1], params[2], params[3]

    y_fit = levy_stable.pdf(x_fit, alpha, beta, gamma, mu)

    #Считаем pvalue - Устойчивое распределение
    r = levy_stable.rvs(alpha, beta, gamma, mu, size=252, random_state=None)

    #Выполняем нормировку набора случайных величин
    norm_rvs_data = get_rvs_intervals(r, bins, x_data)
    return params, r, y_fit, norm_rvs_data

In [14]:
def closer_finding_functions_meixner(x_data, data, columns):
    # #Производим приближение - Распределение Мейснера
    data1 = [[i] for i in data[columns]]
    sample = ot.Sample(data1)
    ot.ResourceMap.SetAsUnsignedInteger('DistributionFactory-DefaultBootstrapSize', 253)
    fittedRes = ot.MeixnerDistributionFactory().build(sample)
    params = fittedRes.getParameter()
    alpha, beta , gamma, delta = params[0], params[1], params[2], params[3]
    distribution = ot.MeixnerDistribution(alpha, beta , gamma, delta)
    y_fit = []
    for x in x_fit:
        y_fit.append(distribution.computePDF(x))
    
    sample = ot.MeixnerDistribution(alpha, beta, gamma, delta).getSample(253)
    r = sample.asPoint()
    norm_rvs_data = get_rvs_intervals(r, bins, x_data)

    return params, distribution, y_fit, norm_rvs_data

### Гиперболическое распределение

In [15]:
p_value_hyperbolic_kstest = []
p_value_hyperbolic_cramervonmises = []
p_value_hyperbolic_anderson = []

for column in dataset.columns:
    emp_data, y_data1, x_fit, x_data, y_data = get_emp_data(dataset, column, 15)
    params_hyperbolic, hyperbolic_distribution, y_fit_hyperbolic, norm_rvs_hyperbolic = closer_finding_functions_hyperbolic(x_data, y_data, dataset, column) 
    p, a, b, mu, delta = params_hyperbolic[0], params_hyperbolic[1], params_hyperbolic[2], params_hyperbolic[3], params_hyperbolic[4]
    p_value_hyperbolic_kstest.append(ks_2samp(norm_rvs_hyperbolic, y_data1, alternative='two-sided', method='auto')[1])
    p_value_hyperbolic_cramervonmises.append(cramervonmises_2samp(norm_rvs_hyperbolic, y_data1, method='auto').pvalue)
    p_value_hyperbolic_anderson.append(anderson_ksamp([norm_rvs_hyperbolic, y_data1]).significance_level)


p_value_hyperbolic = pd.DataFrame(data={'ks_test': p_value_hyperbolic_kstest, 'anderson': p_value_hyperbolic_anderson, 'cramervonmises': p_value_hyperbolic_cramervonmises},
                    index=dataset.columns)

p_value_hyperbolic.to_csv('../p_value/p_value_hyperbolic.csv', index=True)
p_value_hyperbolic

/tmp/ipykernel_4843/3717015333.py:11: UserWarning: p-value floored: true value smaller than 0.001
  p_value_hyperbolic_anderson.append(anderson_ksamp([norm_rvs_hyperbolic, y_data1]).significance_level)


,ks_test,anderson,cramervonmises
log_return_vkco,0.000082,0.001,3.578355e-06
log_return_tcs_il,0.000850,0.001,1.168740e-06
log_return_poly_il,0.002353,0.001,7.488400e-06
log_return_five_il,0.000125,0.001,3.844349e-07


### Stable распределение

In [16]:
p_value_stable_kstest = []
p_value_stable_cramervonmises = []
p_value_stable_anderson = []

for column in dataset.columns:
    params_stable, stable_distribution, y_fit_stable, norm_rvs_stable = closer_finding_functions_stable(x_data, dataset, column) 
    alpha, beta, gamma, delta = params_stable[0], params_stable[1], params_stable[2], params_stable[3],
    p_value_stable_kstest.append(ks_2samp(norm_rvs_stable, y_data1, alternative='two-sided', method='auto')[1])
    p_value_stable_cramervonmises.append(cramervonmises_2samp(norm_rvs_stable, y_data1, method='auto').pvalue)
    p_value_stable_anderson.append(anderson_ksamp([norm_rvs_stable, y_data1]).significance_level)


p_value_stable = pd.DataFrame(data={'ks_test': p_value_stable_kstest, 'anderson': p_value_stable_anderson, 'cramervonmises': p_value_stable_cramervonmises,},
                    index=dataset.columns)
p_value_stable.to_csv('../p_value/p_value_stable.csv', index=True)
p_value_stable


/tmp/ipykernel_4843/3362285898.py:10: UserWarning: p-value capped: true value larger than 0.25
  p_value_stable_anderson.append(anderson_ksamp([norm_rvs_stable, y_data1]).significance_level)


,ks_test,anderson,cramervonmises
log_return_vkco,0.650607,0.250000,0.486867
log_return_tcs_il,0.944728,0.250000,0.525631
log_return_poly_il,0.944728,0.250000,0.465220
log_return_five_il,0.393572,0.173045,0.103757


### Распределение Мейкснера

In [17]:
p_value_meixner_kstest = []
p_value_meixner_cramervonmises = []
p_value_meixner_anderson = []

for column in dataset.columns:
    params_meixner, meixner_distribution, y_fit_meixner, norm_rvs_meixner = closer_finding_functions_meixner(x_data, dataset, column) 
    alpha, beta, gamma, delta = params_meixner[0], params_meixner[1], params_meixner[2], params_meixner[3],
    p_value_meixner_kstest.append(ks_2samp(norm_rvs_meixner, y_data1, alternative='two-sided', method='auto')[1])
    p_value_meixner_cramervonmises.append(cramervonmises_2samp(norm_rvs_meixner, y_data1, method='auto').pvalue)
    p_value_meixner_anderson.append(anderson_ksamp([norm_rvs_meixner, y_data1]).significance_level)


p_value_meixner = pd.DataFrame(data={'ks_test': p_value_meixner_kstest, 'anderson': p_value_meixner_anderson, 'cramervonmises': p_value_meixner_cramervonmises,},
                    index=dataset.columns)
p_value_meixner.to_csv('../p_value/p_value_meixner.csv', index=True)
p_value_meixner

/tmp/ipykernel_4843/2936124394.py:10: UserWarning: p-value capped: true value larger than 0.25
  p_value_meixner_anderson.append(anderson_ksamp([norm_rvs_meixner, y_data1]).significance_level)


,ks_test,anderson,cramervonmises
log_return_vkco,0.286104,0.25,0.259682
log_return_tcs_il,0.989103,0.25,0.674630
log_return_poly_il,0.944728,0.25,0.405749
log_return_five_il,0.944728,0.25,0.445311
